# Data Fusion Contest 2026 - Задача 3 "Герои"
Участникам необходимо подготовить наилучший план маршрутизации с учетом ограничений на временные окна.



## Предыстория задачи
В анализе данных есть много интересных задач, в том числе и не связанных с машинным обучением. Более того, тесно связанная с машинным обучением область оптимизации не ограничивается одними лишь градиентными методами.

Сложные оптимизационные задачи возникают в разных частях бизнеса. В задачах логистики и доставки собирается много данных, используя которые, можно оптимизировать реальные бизнес-процессы “на земле”, с видимым и измеримым снижением затрат.

В этом образовательном соревновании мы приглашаем участников расширить свой кругозор и попробовать свои силы на учебной задаче логистики и маршрутизации. Подобно курьерам и заказам в прикладных VRPTW задачах, участникам необходимо оптимизировать маршрут для героев, собирающих золото на карте приключений.

## Легенда задачи
Начинается новая игровая партия в Героях Меча и Магии, и новая неделя. У вас есть стартовый замок, где вы можете нанимать героев. Вокруг вашего замка начинают появляться водяные мельницы, каждая из которых принесет в казну 500 золота.

Но есть нюанс — хоть этих мельниц и много, но они дают золото только в свой конкретный день недели. Если вы придете к ним позже, то вашу золотую награду уже кто-то заберет, и вы ничего не получите. А если придете раньше, то этой мельницы еще даже не будет на месте. Ваш герой может дождаться того дня, когда она появится, но посетить ее можно только в день ее открытия.

Вам заранее известно где и в какой день появится каждая мельница. У вас даже есть заранее посчитанные расстояния в очках хода ("мувпойнтах") между каждой локацией, а также расстояние от замка с таверной до каждой локации.

На время действия этой недели золота и оптимизации, в игре снимается ограничение на число героев которых вы можете нанять. Вы можете нанять хоть 100 героев одновременно. У каждого героя есть свой запас очков хода. Однако, действует и правило “таверны мечты” — всех героев нужно нанимать последовательно. Если вы знаете что, например, 20-ым героем будет быстрый “логист”, но перед ним в таверне сидят медленные гномы, то увы, придется нанять и гномов.

Это событие с изобилием и водяных мельниц и героев длится ровно одну игровую неделю (7 дней). Как много золота вы успеете собрать своим неограниченным парком героев?

Более подробная информация о механиках, правилах и ограничениях в рамках этой задачи доступна на странице “Задача”.

## Формат решений
Это табличное соревнование с разметкой предоставленного вам .csv файла. Вам необходимо создать алгоритм, способный по предоставленным в рамках соревнования данным, создать новый табличный .csv файл с двумя столбцами:

```text
hero_id, object_id
1, 1
1, 2
...
1, 3
```
- hero_id – идентификатор героя из пула героев;
- object_id – идентификатор объекта с золотом из списка объектов.
Подробности про ограничения на решения можно найти на странице “Задача”.

Данные и пример sample_submit.csv доступны на странице “Данные”.

## Проверка решений
Решения проверяются автоматически путем проверки маршрута на соответствие всем условия постановки задачи и подсчетом полученной итоговой метрики.

Метрика соревнования — VRPTW Score, реализующий подсчет геройской экономики к концу игровой недели:

Где

числозаказов

наградазаобъект

индикатортогочтообъектбылпосещенвегоденьоткрытия

идентификаторгероя

Учитывая условие на строгую последовательность найма героев в таверне, считается что на героев было потрачено столько золота, сколько суммарно героев потребовалось бы нанять вплоть до последнего (с максимальным hero_id в маршрутном решении).

В данной образовательной задаче нет разделения на public/private.


## Оптимизация Маршрутизации Героев (ОМГ)

### Связь с другими VRP задачами
Следуя постановке и легенде соревнования, мы решаем VRPTW-подобную (Vehicle Routing Problem with Time Windows, маршрутизация транспортных средств с ограничениями по времени) задачу оптимизации:

В классическом VRP имеется депо и курьеры, которые развозят заказы в соответствие с матрицей расстояний. В нашем случае в роли депо выступает замок с таверной, в роли курьеров — герои, в роли заказов — водяные мельницы (они же водяные колеса, далее для простоты, будем писать просто мельница), и матрица очков хода (в роли матрицы расстояний/времени).
В классическом VRPTW у заказов появляются временные окна, в рамках которых заказ должен быть доставлен. В дополнение к матрице расстояний добавляется матрица времени (и расчеты идут по ней), а также появляется дополнительный параметр времени обслуживания (время на вручение заказа). В нашем случае в роли временных окон выступают дни доступности мельниц, матрица очков хода в роли роли матрицы времени, и стоимость посещения ("захода") в очках хода.
В дополнение к классическому VRPTW, здесь используется “гетерогенный” парк курьеров. В нашем случае это означает, что у разных героев разный запас ежедневных очков хода. Для простоты считаем что этот ежедневный (восполняемый) запас очков хода у героя не меняется.
В рамках геройской специфики, в задаче действует глобальный счетчик дней (очки хода восполняются на новый день). Есть "правила таверны" на строгую последовательность в найме героев. А также, в задаче действует правило "последнего шага" (ласт-мув).
Как нетрудно догадаться, нам предстоит работать с NP-сложной задачей.

### Данные и их обозначения
Сами данные, их описания и публичные решения доступны на странице “Данные”.

Для формулирования всех правил и условий, стоит задать рамку и описания самих данных:

data_heroes.csv — данные о героях. Всего героев можно нанять до 100 штук. Содержит столбцы hero_id (идентификатор героя, int от 1 до 100) и move_points (дневной запас очков хода, int от 1300 до 1900).
data_objects.csv — данные о золотоносных объектах (водяных мельницах). Всего мельниц 700 штук. Содержит столбцы object_id (идентификатор объекта, int от 1 до 700), day_open (день “появления” объекта когда он дает награду, int от 1 до 7), и reward (награда за посещение, int и в наших данных для всех равен 500).
dist_start.csv — данные о расстояниях от замка (депо) до каждой из мельниц (объектов/заказов) в очках хода. Содержит 700 строк и 2 столбца:  object_id (идентификатор объекта, int от 1 до 700) и dist_start (расстояние в очках хода, int).
dist_objects.csv — данные о попарных расстояниях между каждой из мельниц в очках хода. Матрица 700 х 700 с header-ом, все значения целочисленные (int).
Глобальные параметры для задачи:

visit_cost = 100 — стоимость посещения мельницы в очках хода (аналогично времени обслуживания).
hero_cost = 2500 — стоимость найма героя в таверне (используется для расчета метрики).
## Условия, правила и ограничения на маршруты

### VRP - совместимые условия
1. Все герои начинают в стартовом замке/таверне (депо)
- Расстояния от замка до каждой из мельниц определяется dist_start из dist_start.scv.
- Замок считается объектом 0 в маршруте. Его не требуется напрямую указывать в своих маршрутах.

2. Маршрутизация героев
- Все герои должны передвигаться исключительно по дискретному маршруту между замком и мельницами.
- Иные перемещения помимо заранее заданного списка мельниц (объектов) запрещены и не рассматриваются.

3. Уникальность посещений
- Каждую мельницу (объект) можно посетить не более одного раза за все дни маршрута всех героев.
4. Число героев
- В отличие от оригинальной игры, вы можете использовать любое количество героев из data_heroes.csv, хоть  всех 100, с первого дня.

5. Увольнение героев
- Если вам понадобится избавиться от героя, это можно сделать, добавив в его маршрут объект 0 (он же замок/депо).
- Увольнение происходит моментально, с сжиганием всех остававшихся у героя очков хода.
- Уволенные герои уходят навсегда и больше не участвуют в игре.
- Увольнение опционально и в рамках базовой постановки не обязательно.

### Тематические VRPTW условия
6. Глобальная смена дней
- Аналогично игре, счетчик дней глобален для всех задействованных героев.
- В начале дня все герои полностью восполняют свои очки хода move_points.
- День заканчивается в тот момент, когда каждый из героев исчерпал свои очки хода (будь то завершив путь “последним шагом”, “заночевав” перед ожидаемой мельницей,  будучи “уволенным”, либо оказавшись посреди пути к следующей мельнице).
- Сами глобальные дни работают как временные окна в VRPTW задачах.

7. Очки хода героев
- У каждого героя есть свой запас очков хода move_points, у кого-то он больше, у кого-то меньше (медиана 1560).
- В начале дня запас очков хода полностью восстанавливается. Изменения в восстанавливаемом числе очков хода (как посещения конюшен или водопоев) не предусмотрены.
- В конце дня неиспользованные очки хода либо сгорают, либо используются в соответствие с условиями посещения “последним шагом” или приездом заранее.

8. Путешествие героев
- Перемещение от одной мельницы к другой отнимает очки хода в соответствии с матрицей  расстояний dist_objects.csv.
- Если остающихся очков хода не хватает для достижения мельницы текущим днем, мы считаем что герой прошел ровно столько очков сколько у него оставалось в данный момент (и закончил свой ход с 0 остающимися очками хода). Недостающий остаток будет вычтен из путешествия до интересующей героя мельницы уже на следующий день.
- Подобный перенос очков хода с текущего дня на следующий работает автоматически. Изменение своего маршрута на полпути между объектами на старте следующего дня запрещено.

### VRPTW - совместимые условия
9. Стоимость посещения мельницы (объекта)
- При посещении мельницы, при условии что она уже появилась, всегда стоит ровно visit_cost = 100 очков хода.
- В рамках этой задачи действует специальное исключение (пункт 14) про правило “последнего шага”. Если у героя осталось хотя бы одно очко хода, но ему не хватает всех 100 очков для посещения, то мы считаем что посещение успешно состоялось и герой завершает ход с 0 очков хода.

10. Раннее прибытие к мельнице (объекту) и ожидание на месте
- Если герой прибыл к месту появления мельницы заранее (до ее day_open), он принудительно стоит на месте и ожидает появления мельницы.
- Пока герой ждёт, его оставшиеся очки хода сгорают. Это относится и к очкам хода за текущий день, и к очкам хода последующих дней, в случае если мельница появится не на следующий день, а через 1 или более.
- То есть, герой “ночует”, стоя на месте ровно до наступления дня day_open.
- После ожидания, очки хода полностью восстанавливаются, с героя списываются visit_cost очков хода за посещение, и начисляется награда reward за успешное посещение мельницы.

11. Своевременное прибытие к мельнице (объекту)
- Если герой прибывает к мельнице вовремя (тем же днем что и day_open), то с него списывается visit_cost очков хода, начисляется награда reward, и герой отправляется дальше по своему маршруту к следующей мельнице.

12. Позднее прибытие к мельнице (объекту)
- Если герой прибывает к мельнице с опозданием (днём позднее чем day_open), то с него все равно списывается visit_cost очков хода, но награда не начисляется. Опоздавший герой герой отправляется дальше по своему маршруту.
- Учитывая уникальность посещений (пункт 3), в общем маршруте всех героев данное посещение просто сожгло очки хода, хотя могло бы быть посещено другим героем вовремя.

### Специальные геройские условия
13. Последовательный найм геров ("правила таверны")
- Герои нанимаются по порядку, в последовательности их hero_id.
- Это значит, что если у вас пока есть только герой 1, и вы хотите вывести собирать золото героя 3, то вам все равно нужно нанять героя 2. Вы можете отправить героя 2 по своему маршруту, либо сразу же уволить его. Но чтобы нанять героя 3, вам все равно нужно потратить золото и на найм героя 2.
- В рамках условностей, можно считать что все ваши нанятые герои стартуют в первый день, но покидают таверну ровно в день появления day_open их первой мельницы (не вылезают из таверны пока дорога не позовет).
- Учитывая это правило, в итоговой метрике вашего собранного золота, достаточно знать максимальный идентификатор героя hero_id(так как вам все равно пришлось бы нанять всех предшествующих ему).
- Это условие реализует правила геройской таверны. Если вам встретилась “таверна мечты” с Уфретином и Стракером, вам все равно придется тратить золото.

14. Посещение "последним шагом" (на-ласт-мув)
- Если у героя осталось от 1 до 100 очков хода и посещение мельницы еще возможно (но на само посещение очков хода не хватает), то посещение считается успешным, а очков на конец хода остается ровно 0.
- Ранее посещение мельниц (объектов) последним шагом умышленно запрещается. Мы считаем что мельница еще не появилась, поэтому и зайти в нее даже на последний шаг нельзя.
- Это условие реализует геройскую механику с легендарными “на-ласт-мувы” действиями героев.

15. Конец игровой недели и подведение итогов
- Игровая неделя заканчивается в конце седьмого дня.
- Все что будет происходить после не имеет значения, так как доступных мельниц после 7 дня больше нет.
- Можно считать что все герои тоже исчезают, но это не имеет значения в рамках метрики задачи соревнования.
- Как и в игре в Герои, на все-про-все (геройский VRPTW) ровно 1 неделя.

## Метрика и проверка решений
Официальная метрика соревнования называется VRPTW Score. Вы можете читать ее как Gold Score — сколько золота вам удастся собрать за игровую неделю, за вычетом суммарного золота, потраченного на найм героев.

Где

наградазаобъект

индикатортогочтомельницабылапосещенавденьеепоявления

идентификаторгероя

## Ограничение на маршруты (решения)
В качестве решений принимаются .csv файлы с 2 столбцами: hero_id и object_id.
Можно использовать только те id, которые есть в предоставленных файлах. hero_id от 1 до 100, object_id от 1 до 700 (0 считаем за депо, его явно указывать в качестве первого объекта не нужно). Все строки с невалидными id удаляются.
Каждый object_id может использоваться ровно 1 раз. Все строки-дубликаты удаляются.
Порядок посещения object_id важен и является основой задачи. Осторожнее работайте с сортировкой строк.
В отличие от классического VRPTW, в ваших решениях разрешено игнорировать мельницы (аналог заказов). В большинстве реальных приложений это запрещается, но в рамках учебной геройской задачи здесь это разрешено.

## Данные
Для решения задачи 3 “Герои” предлагается 4 основных набора данных:

0.7KB, базовая информация о героях и их дневном количестве очков хода

6.9KB, базовая информация об объектах (водяных мельницах), их дней открытия и полагающейся награде

5.5KB, данные о расстояниях от депо (замка/таверны) до каждого из объектов (мельниц)

2.0MB, данные о попарных расстояниях между объектами (мельницами)

## Материалы для решения
4.7KB, валидный пример базового решения на основе жадного алгоритма

12.3KB, файл с классов HeroesInstance с утилитами для создания и проверки собственных решений

51.5KB, ноутбук с примером использования предоставленных утилит для создания "жадного" базового решения

## Публичные решения
Вы можете пополнить этот список в спецноминации Companion :)

Приветствуются самые разные материалы:

Визуализация работы планировщиков и метрик по ходу и поиска и выполнения маршрутов. Ноутбуки с геройской тематической визуализацией также приветствуется.
Ноутбуки и решения на основе и актуальных планировщиков и научных подходов.
Самостоятельное решение усложненных версий задачи. Например с честными 8/9 героям в любой момент времени; Capacitated постановка с лимитом на число объектов (в день/на героя); и т.д.
Материалы с вводного митапа: появятся после вводного митапа 19 февраля :)

### Глоссарий данных о героях data_heroes.csv
- hero_id – id героя, int от 1 до 100
move_points — дневной запас очков хода героя, int, от 1300 до 1900
### Глоссарий данных об объектах data_objects.csv
- object_id – id объекта (водяной мельницы), int от 1 до 700
day_open — день “появления” объекта когда он дает награду, int от 1 до 7
reward — награда за посещение объекта, int, в базовой постановке задачи для всех равен 500
### Глоссарий данных о стартовых расстояниях dist_start.csv
- object_id – id объекта, int от 1 до 700
dist_start — расстояние от депо (замка) до объекта, int
### Глоссарий данных о стартовых расстояниях dist_objects.csv
object_i – у матрицы расстояний есть только заголовок, но нет имен строк


### 1. Imports and Configuration

Defines dependencies and core constants (`DATA_DIR`, costs, and search settings).


In [1]:
# Baseline for Data Fusion Contest 2026 - Task 3 "Heroes"
# CPU-only, no external solvers. Designed for running locally in a Jupyter notebook (.ipynb).

from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Iterable, Set

import numpy as np
import pandas as pd


# =========================
# Config
# =========================

# !!! CHANGE THIS PATH !!!
DATA_DIR = Path(r"D:\Train Data\data fusion 2026 track3")

VISIT_COST = 100
HERO_COST = 2500

RANDOM_SEED = 42
MAX_HERO_ID = 100

# Search settings: how many randomized attempts per K (keep small for baseline)
N_SEEDS_PER_K = 1  # set to 3..10 if you want a bit better baseline at the cost of time


### 2. Data Loading Utilities

Loads all official CSV files and validates expected schema and dimensions.


In [2]:
# =========================
# Data loading
# =========================

def _read_csv(path: Path, **kwargs) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    return pd.read_csv(path, **kwargs)

def load_data(data_dir: Path):
    heroes = _read_csv(data_dir / "data_heroes.csv")
    objects = _read_csv(data_dir / "data_objects.csv")
    dist_start_df = _read_csv(data_dir / "dist_start.csv")
    dist_obj_df = _read_csv(data_dir / "dist_objects.csv")

    # Validate basic columns
    for col in ["hero_id", "move_points"]:
        if col not in heroes.columns:
            raise ValueError(f"data_heroes.csv must contain column '{col}'")
    for col in ["object_id", "day_open", "reward"]:
        if col not in objects.columns:
            raise ValueError(f"data_objects.csv must contain column '{col}'")
    for col in ["object_id", "dist_start"]:
        if col not in dist_start_df.columns:
            raise ValueError(f"dist_start.csv must contain column '{col}'")

    # Build mappings
    hero_mp = np.zeros(MAX_HERO_ID + 1, dtype=np.int32)
    for r in heroes.itertuples(index=False):
        hid = int(r.hero_id)
        hero_mp[hid] = int(r.move_points)

    obj_day = np.zeros(701, dtype=np.int32)
    obj_reward = np.zeros(701, dtype=np.int32)
    for r in objects.itertuples(index=False):
        oid = int(r.object_id)
        obj_day[oid] = int(r.day_open)
        obj_reward[oid] = int(r.reward)

    dist_start = np.zeros(701, dtype=np.int32)
    for r in dist_start_df.itertuples(index=False):
        oid = int(r.object_id)
        dist_start[oid] = int(r.dist_start)

    # dist_objects.csv can sometimes be saved with an extra unnamed index column
    dist_obj_df = dist_obj_df.copy()
    if dist_obj_df.shape[1] == 701:
        first = str(dist_obj_df.columns[0]).lower()
        if first.startswith("unnamed") or first in {"object_i", "object_id", "0"}:
            dist_obj_df = dist_obj_df.iloc[:, 1:]

    if dist_obj_df.shape != (700, 700):
        raise ValueError(
            f"dist_objects.csv expected 700x700 matrix, got {dist_obj_df.shape}. "
            f"Please inspect the file formatting."
        )

    dist_obj = dist_obj_df.to_numpy(dtype=np.int32)

    # pad to 1-based indexing: dist_obj_p[i, j] for i,j in 1..700
    dist_obj_p = np.zeros((701, 701), dtype=np.int32)
    dist_obj_p[1:, 1:] = dist_obj

    return hero_mp, obj_day, obj_reward, dist_start, dist_obj_p


### 3. Greedy Route Builder

Builds day-level greedy routes and converts them into submission format.


In [3]:
# =========================
# Baseline planner (day-only routes)
# =========================

def greedy_route_one_day(
    candidates: Iterable[int],
    hero_move_points: int,
    dist_start: np.ndarray,
    dist_obj: np.ndarray,
    rng: np.random.Generator,
) -> List[int]:
    """
    Build a simple nearest-neighbor route for a single day:
    - start at depot (0)
    - repeatedly go to the nearest reachable object
    - apply VISIT_COST; allow last-move visit if remaining in [1..99]

    This does NOT mutate input candidates.
    """
    remaining: Set[int] = set(int(x) for x in candidates)
    route: List[int] = []

    cur = 0
    mp = int(hero_move_points)

    while remaining and mp > 0:
        feasible = []
        best_d = None

        # find nearest reachable (need to arrive with at least 1 mp)
        for oid in remaining:
            d = int(dist_start[oid]) if cur == 0 else int(dist_obj[cur, oid])
            if d <= mp - 1:
                if best_d is None or d < best_d:
                    best_d = d
                    feasible = [oid]
                elif d == best_d:
                    feasible.append(oid)

        if best_d is None:
            break

        # tie-break randomly among equal distances (helps a little across seeds)
        nxt = feasible[rng.integers(0, len(feasible))] if len(feasible) > 1 else feasible[0]

        # travel
        mp_after_travel = mp - best_d

        # visit
        if mp_after_travel >= VISIT_COST:
            mp = mp_after_travel - VISIT_COST
            route.append(nxt)
            remaining.remove(nxt)
            cur = nxt
        else:
            # last-move: mp_after_travel in [1..99]
            route.append(nxt)
            remaining.remove(nxt)
            mp = 0
            break

    return route


def build_baseline_solution(
    max_hero_id_allowed: int,
    hero_mp: np.ndarray,
    obj_day: np.ndarray,
    dist_start: np.ndarray,
    dist_obj: np.ndarray,
    rng: np.random.Generator,
) -> Dict[int, List[int]]:
    """
    Assign each hero (1..max_hero_id_allowed) to exactly one day,
    picking the day where this hero can collect the most remaining mills (greedy preview),
    then committing the route and removing those mills.
    """
    remaining_by_day: Dict[int, Set[int]] = {d: set() for d in range(1, 8)}
    for oid in range(1, 701):
        d = int(obj_day[oid])
        if 1 <= d <= 7:
            remaining_by_day[d].add(oid)

    routes: Dict[int, List[int]] = {}

    for hid in range(1, max_hero_id_allowed + 1):
        mp = int(hero_mp[hid])
        if mp <= 0:
            continue

        best_day: Optional[int] = None
        best_route: List[int] = []

        # Try each day, select the day with maximal route length
        # (reward is constant 500 in baseline, so length ~ reward).
        for d in range(1, 8):
            if not remaining_by_day[d]:
                continue
            preview_route = greedy_route_one_day(
                candidates=remaining_by_day[d],
                hero_move_points=mp,
                dist_start=dist_start,
                dist_obj=dist_obj,
                rng=rng,
            )
            if len(preview_route) > len(best_route):
                best_route = preview_route
                best_day = d

        if best_day is None or len(best_route) == 0:
            # hero not used; but we may still use later heroes (with higher move points)
            continue

        routes[hid] = best_route
        for oid in best_route:
            remaining_by_day[best_day].discard(oid)

    return routes


def routes_to_submit_df(routes: Dict[int, List[int]]) -> pd.DataFrame:
    rows = []
    for hid in sorted(routes.keys()):
        for oid in routes[hid]:
            rows.append((hid, oid))
    return pd.DataFrame(rows, columns=["hero_id", "object_id"])


### 4. Simulator and Scorer

Simulates route execution under contest rules and computes `VRPTW Score`.


In [4]:
# =========================
# Simulator / scorer (general, supports multi-day travel + early waiting + last-move)
# =========================

@dataclass
class HeroState:
    hero_id: int
    move_points: int
    route: List[int]
    idx: int = 0               # next object index to VISIT (not just reach)
    loc: int = 0               # current node: 0 or object_id
    traveling_to: Optional[int] = None
    travel_remaining: int = 0
    waiting_until: Optional[int] = None
    waiting_object: Optional[int] = None
    start_day: int = 1         # day when hero leaves tavern (day_open of first object)
    active: bool = True

def simulate_and_score(
    submit_df: pd.DataFrame,
    hero_mp: np.ndarray,
    obj_day: np.ndarray,
    obj_reward: np.ndarray,
    dist_start: np.ndarray,
    dist_obj: np.ndarray,
    max_day: int = 7,
) -> Tuple[int, Dict[str, int]]:
    """
    Simulate according to rules described on the competition page:
    - global days 1..7
    - heroes have daily move_points, restored each day
    - travel can span multiple days
    - early arrival forces waiting until day_open (burning remaining mp in those days)
    - last-move visit allowed if day >= day_open and remaining mp in [1..99] for paying VISIT_COST
    - metric counts only visits on exactly day_open, and subtracts HERO_COST * max(hero_id)
    """
    if submit_df.empty:
        return 0, {
            "score": 0,
            "gold_on_time": 0,
            "visits_on_time": 0,
            "visits_total": 0,
            "max_hero_id": 0,
            "hero_cost": 0,
        }

    # Basic cleaning similar to described constraints
    df = submit_df.copy()
    df = df[(df["hero_id"].between(1, 100)) & (df["object_id"].between(1, 700))]
    # Drop duplicate object assignments (keep first occurrence in file order)
    df = df.drop_duplicates(subset=["object_id"], keep="first")

    if df.empty:
        return 0, {
            "score": 0,
            "gold_on_time": 0,
            "visits_on_time": 0,
            "visits_total": 0,
            "max_hero_id": 0,
            "hero_cost": 0,
        }

    max_hero_id_used = int(df["hero_id"].max())
    hero_cost_total = HERO_COST * max_hero_id_used

    # Build per-hero route preserving file order
    routes: Dict[int, List[int]] = {}
    for hid, g in df.groupby("hero_id", sort=False):
        routes[int(hid)] = [int(x) for x in g["object_id"].tolist()]

    # Initialize hero states
    heroes: Dict[int, HeroState] = {}
    for hid, route in routes.items():
        if len(route) == 0:
            continue
        mp = int(hero_mp[hid])
        if mp <= 0:
            continue
        first_obj = route[0]
        sd = int(obj_day[first_obj])
        # If day_open is invalid, default to day1
        if sd < 1 or sd > 7:
            sd = 1
        heroes[hid] = HeroState(
            hero_id=hid,
            move_points=mp,
            route=route,
            start_day=sd,
        )

    visited_on_time = 0
    visited_total = 0
    gold_on_time = 0
    visited_flag = np.zeros(701, dtype=np.int8)

    def distance(a: int, b: int) -> int:
        if a == 0:
            return int(dist_start[b])
        return int(dist_obj[a, b])

    for day in range(1, max_day + 1):
        for hid in sorted(heroes.keys()):
            h = heroes[hid]
            if not h.active:
                continue
            if h.idx >= len(h.route):
                h.active = False
                continue

            # before start_day, hero stays in tavern
            if day < h.start_day:
                continue

            mp = h.move_points

            # waiting logic
            if h.waiting_until is not None and h.waiting_object is not None:
                if day < h.waiting_until:
                    # forced waiting burns the day
                    continue
                elif day == h.waiting_until:
                    # visit happens at day start (no travel)
                    oid = h.waiting_object
                    open_day = int(obj_day[oid])

                    # mill has "appeared" now
                    # pay visit cost
                    mp -= VISIT_COST
                    if mp < 0:
                        mp = 0

                    # register visit
                    visited_total += 1
                    visited_flag[oid] = 1
                    if day == open_day:
                        visited_on_time += 1
                        gold_on_time += int(obj_reward[oid])

                    # clear waiting state and advance route
                    h.waiting_until = None
                    h.waiting_object = None
                    h.loc = oid
                    h.idx += 1

                    if h.idx >= len(h.route) or mp <= 0:
                        continue

            # if not currently traveling and not waiting, set travel target to next object
            if h.traveling_to is None and h.idx < len(h.route):
                tgt = h.route[h.idx]
                # (we do not use firing with object 0 in this baseline; support could be added)
                h.traveling_to = tgt
                h.travel_remaining = distance(h.loc, tgt)

            # execute moves while there is mp
            while mp > 0 and h.idx < len(h.route):
                tgt = h.route[h.idx]

                # ensure travel target matches route[idx]
                if h.traveling_to != tgt:
                    h.traveling_to = tgt
                    h.travel_remaining = distance(h.loc, tgt)

                # travel
                if h.travel_remaining > mp:
                    h.travel_remaining -= mp
                    mp = 0
                    break
                else:
                    mp -= h.travel_remaining
                    h.travel_remaining = 0
                    h.loc = tgt
                    h.traveling_to = None

                # arrived to target with current mp
                open_day = int(obj_day[tgt])

                # early arrival -> forced waiting (no visit now)
                if day < open_day:
                    h.waiting_until = open_day
                    h.waiting_object = tgt
                    # remaining mp is burned
                    mp = 0
                    break

                # day >= open_day -> can attempt to visit
                # note: even if mp==0, cannot visit today; will try next day (late potentially)
                if mp >= VISIT_COST:
                    mp -= VISIT_COST
                    # mark visit now (if not visited before; duplicates are removed earlier so safe)
                    if visited_flag[tgt] == 0:
                        visited_total += 1
                        visited_flag[tgt] = 1
                        if day == open_day:
                            visited_on_time += 1
                            gold_on_time += int(obj_reward[tgt])
                    h.idx += 1
                elif 1 <= mp < VISIT_COST:
                    # last-move: successful visit, mp becomes 0
                    if visited_flag[tgt] == 0:
                        visited_total += 1
                        visited_flag[tgt] = 1
                        if day == open_day:
                            visited_on_time += 1
                            gold_on_time += int(obj_reward[tgt])
                    h.idx += 1
                    mp = 0
                    break
                else:
                    # mp == 0: cannot visit today, will attempt next day (standing at the object)
                    mp = 0
                    break

                # if route ended, burn remaining mp
                if h.idx >= len(h.route):
                    mp = 0
                    break

                # else prepare next travel (still same day if mp>0)
                if mp > 0:
                    nxt = h.route[h.idx]
                    h.traveling_to = nxt
                    h.travel_remaining = distance(h.loc, nxt)

    score = int(gold_on_time - hero_cost_total)
    stats = {
        "score": score,
        "gold_on_time": int(gold_on_time),
        "visits_on_time": int(visited_on_time),
        "visits_total": int(visited_total),
        "max_hero_id": int(max_hero_id_used),
        "hero_cost": int(hero_cost_total),
    }
    return score, stats


### 5. End-to-End Run

Searches over hero limits, keeps the best solution, and writes `... .csv`.


In [5]:
# =========================
# Full baseline run: pick best K, write submit
# =========================

hero_mp, obj_day, obj_reward, dist_start, dist_obj = load_data(DATA_DIR)

best = {
    "score": -10**18,
    "K": None,
    "seed": None,
    "df": None,
    "stats": None,
}

base_rng = np.random.default_rng(RANDOM_SEED)

for K in range(1, MAX_HERO_ID + 1):
    for s in range(N_SEEDS_PER_K):
        rng = np.random.default_rng(int(base_rng.integers(0, 10**9)))

        routes = build_baseline_solution(
            max_hero_id_allowed=K,
            hero_mp=hero_mp,
            obj_day=obj_day,
            dist_start=dist_start,
            dist_obj=dist_obj,
            rng=rng,
        )
        submit_df = routes_to_submit_df(routes)

        score, stats = simulate_and_score(
            submit_df=submit_df,
            hero_mp=hero_mp,
            obj_day=obj_day,
            obj_reward=obj_reward,
            dist_start=dist_start,
            dist_obj=dist_obj,
            max_day=7,
        )

        if score > best["score"]:
            best.update({
                "score": score,
                "K": K,
                "seed": None,
                "df": submit_df,
                "stats": stats
            })

# Save best submission
out_path = Path.cwd() / "svg_heroes.csv"
best["df"].to_csv(out_path, index=False)

print(f"Best K tried: {best['K']}")
print("Stats:", best["stats"])
print(f"Saved: {out_path}")
print("Head:")
print(best["df"].head(10))
print("Rows:", len(best["df"]))


Best K tried: 50
Stats: {'score': 32500, 'gold_on_time': 157500, 'visits_on_time': 315, 'visits_total': 315, 'max_hero_id': 50, 'hero_cost': 125000}
Saved: d:\Apps\PyCharm\pythonProject\svg_heroes.csv
Head:
   hero_id  object_id
0        1        183
1        1        509
2        1        537
3        1        226
4        1        651
5        1        505
6        1        599
7        1        387
8        2        428
9        2        427
Rows: 315


In [6]:
'''
Final VRPTW Score on public ds = 294 500
At the time of submitting the solution it was 1st place on leaderboard
Run on CPU
'''

'\nFinal VRPTW Score on public ds = 294 500\nAt the time of submitting the solution it was 1st place on leaderboard\nRun on CPU\n'